In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Importing Necessary Libraries

In [ ]:
import nltk, re, time
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import random

Importing Data

In [ ]:
data = pd.read_csv("../input/placement_survey_data.csv")
data.head()

Choosing Required Columns

In [ ]:
data = data[['Feedback regarding all placement related activities conducted till date.','How would you rate the placement department?']]
data.columns = ['review','liked']
data.head()

Binarizing Class

In [ ]:
#data['liked'] = (data['liked'] > 2).astype(int)
#data.head()

Cleaning Data

In [ ]:
corpus = []
for i in range(len(data.index)):
    review = re.sub('[^a-zA-Z]', ' ', data['review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

Tokenization

In [ ]:
cv = CountVectorizer(max_features = 252) #choosing top 'n' most frequent tokens
X = cv.fit_transform(corpus).toarray()
y = data.iloc[:,-1].values
np.asarray(X.sum(axis=0)) #frequency of tokens

Splitting Dataset into Train and Test

In [ ]:
indices = range(len(data.index))
X_train, X_test,y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size = 0.3, random_state = random.randint(0, 1000))

Feature Scaling

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(float))
X_test = sc.transform(X_test.astype(float))

Training the Model

In [ ]:
clf = MLPClassifier(solver='sgd', activation = 'tanh', alpha=1e-5, hidden_layer_sizes=(64, 64, 64, 32, 16, 4, 4, 4), learning_rate_init = 0.15, learning_rate = 'adaptive', max_iter = 10000, random_state = random.randint(0, 1000), verbose = False, shuffle = True, early_stopping = True)
clf.fit(X_train, y_train)

Testing Accuracy

In [ ]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
d = {'test': y_test, 'pred': y_pred}
testpred = pd.DataFrame(data=d)
testpred.insert(0, column = 'review', value = data['review'][indices_test].values)
testpred['test'] = (testpred['test'] > 2).astype(int)
testpred['pred'] = (testpred['pred'] > 2).astype(int)
print(testpred.head())
print(accuracy_score(testpred['test'], testpred['pred']))
testpred.to_csv("output.csv")